# Population Density

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [2]:
pwd

'/home/jovyan/work/Desktop/UCL/Term 2/UCL Dissertation/Dissertation_R'

In [14]:
population_gdf = gpd.read_file('/home/jovyan/work/Desktop/UCL/Term 2/UCL Dissertation/Dissertation_R/data/lsoa/lsoa_pop.geojson')
population_gdf.to_crs(epsg = 27700)
population_gdf.head(1)

,LSOA11CD,lsoa_name,LSOA11NM,shape_area,all_ages,pop_density,geometry
0,E01000001,City of London 001A,City of London 001A,129865.314354,1749.0,13467.799379,"POLYGON ((532162.491 181867.763, 532248.394 18..."


In [16]:
print(population_gdf.crs)

epsg:27700


In [11]:
grid_gdf = gpd.read_file('/home/jovyan/work/Desktop/UCL/Term 2/UCL Dissertation/Dissertation_R/data/locomizer/Greater_London.json')
grid_gdf.head(1)

,_index,city,index_h3_10lvl,geometry
0,8a195da519a7fff,Greater London,8a195da519a7fff,"POLYGON ((-0.25164 51.58819, -0.25260 51.58802..."


In [18]:
grid_gdf = grid_gdf.to_crs(epsg = 27700)
grid_gdf.crs

<Derived Projected CRS: EPSG:27700>
Name: OSGB36 / British National Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: United Kingdom (UK) - offshore to boundary of UKCS within 49°45'N to 61°N and 9°W to 2°E; onshore Great Britain (England, Wales and Scotland). Isle of Man onshore.
- bounds: (-9.0, 49.75, 2.01, 61.01)
Coordinate Operation:
- name: British National Grid
- method: Transverse Mercator
Datum: Ordnance Survey of Great Britain 1936
- Ellipsoid: Airy 1830
- Prime Meridian: Greenwich

In [22]:
population_gdf['oa_area'] = population_gdf.area
population_gdf.head(1)

,LSOA11CD,lsoa_name,LSOA11NM,shape_area,all_ages,pop_density,geometry,oa_area
0,E01000001,City of London 001A,City of London 001A,129865.314354,1749.0,13467.799379,"POLYGON ((532162.491 181867.763, 532248.394 18...",133324.9787


In [23]:
# Projecting the population data onto the hexagonal grid

intersections = gpd.overlay(grid_gdf, population_gdf, how='intersection')

intersections['oa_ratio'] = intersections.area / intersections.oa_area

intersections['population'] = intersections.all_ages * intersections.oa_ratio

intersections = intersections[['_index', 'population']].groupby('_index').sum().sort_values(by='population', ascending=False)

In [29]:
# merge two data
intersections = gpd.GeoDataFrame(grid_gdf[['_index', 'geometry']].merge(intersections, on='_index'))

In [30]:
intersections.head(1)

,_index,geometry,population
0,8a195da519a7fff,"POLYGON ((521219.724 189151.929, 521153.655 18...",97.096838


In [31]:
# Save the GeoDataFrame as a GeoJSON file
intersections.to_file('hex_pop.geojson', driver='GeoJSON')

# Projecting population at hexagon-level to neighbourhood-level

In [43]:
grid_gdf = gpd.read_file('/home/jovyan/work/Desktop/UCL/Term 2/UCL Dissertation/Dissertation_R/hex_pop.geojson')
grid_gdf.to_crs(epsg = 27700)
grid_gdf.head(1)

,_index,population,geometry
0,8a195da519a7fff,97.096838,"POLYGON ((521219.724 189151.929, 521153.655 18..."


In [44]:
neighbourhood_gdf = gpd.read_file('/home/jovyan/work/Desktop/UCL/Term 2/UCL Dissertation/Dissertation_R/data/neighbourhood/whole_neighbourhoods_polygon_500.geojson')
neighbourhood_gdf.to_crs(epsg = 27700)
neighbourhood_gdf.head(1)

,nb_clus,geometry
0,9,"POLYGON ((518492.330 170869.907, 518651.214 17..."


In [45]:
# Calculate the area of each hexagon
grid_gdf['hexagon_area'] = grid_gdf.area

In [46]:
# Projecting the hexagon level population data onto the neighborhood polygons
intersections = gpd.overlay(grid_gdf, neighbourhood_gdf, how='intersection')

In [47]:
# Calculate the ratio of intersection area to original hexagon area
intersections['hexagon_ratio'] = intersections.area / intersections.hexagon_area

In [48]:
intersections.head(3)

,_index,population,hexagon_area,nb_clus,geometry,hexagon_ratio
0,8a195da519a7fff,97.096838,13506.572879,53838,"POLYGON ((521275.445 189101.184, 521265.099 18...",1.0
1,8a195da5574ffff,145.898344,13505.473659,53838,"POLYGON ((521459.311 189499.300, 521448.965 18...",1.0
2,8a195da556e7fff,144.213610,13506.621289,53838,"POLYGON ((521062.906 189375.541, 521052.558 18...",1.0


In [49]:
# Calculate the population in each intersection and sum by neighborhood ID
intersections['population'] = intersections.population * intersections.hexagon_ratio
neighborhood_population = intersections.groupby('nb_clus')['population'].sum().reset_index()

In [50]:

# Merge the neighborhood population data back to the neighborhood polygons
neighbourhood_gdf = neighbourhood_gdf.merge(neighborhood_population, on='nb_clus')


In [52]:
max(neighbourhood_gdf.population)

125246.20302925151